# Adv_GAN
a Pytorch implementation of the paper "Generating Adversarial Examples with Adversarial Networks" (advGAN).

## 1. 导入一些第三方库

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.datasets
import torchvision.transforms as transforms


## 2. 待攻击的目标模型

### 2.1 定义目标模型的类

In [0]:
class MNIST_target_net(nn.Module):
    
    def __init__(self):
        super(MNIST_target_net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*4*4, 200)
        self.fc2 = nn.Linear(200, 200)
        self.logits = nn.Linear(200, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*4*4)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = F.relu(self.fc2(x))
        x = self.logits(x)
        return x

### 2.2 产生目标模型

In [7]:
use_cuda = True
image_nc = 1
model_batch_size = 256

# define what device we are using

print("CUDA Avaliable: ", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Avaliable:  True


### 2.3 下载minist数据集

In [8]:
mnist_dataset = torchvision.datasets.MNIST('./dataset', 
                                           train=True, 
                                           transform=transforms.ToTensor(), 
                                           download=True 
                                          )
train_dataloader = DataLoader(mnist_dataset,
                              batch_size=model_batch_size, 
                              shuffle=False, 
                              num_workers=1)


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw



Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw



Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw



Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw
Processing...
Done!


### 2.4 开始训练目标模型

In [9]:
# training the target model
target_model = MNIST_target_net().to(device)
target_model.train()
opt_model = torch.optim.Adam(target_model.parameters(), lr=0.001)
epochs = 40
for epoch in range(epochs):
    loss_epoch = 0
    if epoch == 20:
        opt_model = torch.optim.Adam(target_model.parameters(), lr=0.0001)
    for i, data in enumerate(train_dataloader, 0):
        train_imgs, train_labels = data
        train_imgs, train_labels = train_imgs.to(device), train_labels.to(device)
        logits_model = target_model(train_imgs)
        loss_model = F.cross_entropy(logits_model, train_labels)
        loss_epoch += loss_model
        opt_model.zero_grad()
        loss_model.backward()
        opt_model.step()

    print('loss in epoch %d: %f' % (epoch, loss_epoch.item()))

loss in epoch 0: 101.177719
loss in epoch 1: 26.683292
loss in epoch 2: 17.823374
loss in epoch 3: 14.675725
loss in epoch 4: 11.997814
loss in epoch 5: 9.868692
loss in epoch 6: 8.840528
loss in epoch 7: 7.704131
loss in epoch 8: 6.996535
loss in epoch 9: 6.211140
loss in epoch 10: 6.042154
loss in epoch 11: 5.649035
loss in epoch 12: 4.948034
loss in epoch 13: 4.428883
loss in epoch 14: 4.390535
loss in epoch 15: 4.021057
loss in epoch 16: 3.904166
loss in epoch 17: 3.712560
loss in epoch 18: 2.778519
loss in epoch 19: 3.153136
loss in epoch 20: 1.937186
loss in epoch 21: 1.355775
loss in epoch 22: 1.124467
loss in epoch 23: 1.223719
loss in epoch 24: 1.018934
loss in epoch 25: 0.816988
loss in epoch 26: 0.789737
loss in epoch 27: 0.580767
loss in epoch 28: 0.597177
loss in epoch 29: 0.604954
loss in epoch 30: 0.493919
loss in epoch 31: 0.601661
loss in epoch 32: 0.542152
loss in epoch 33: 0.489124
loss in epoch 34: 0.465762
loss in epoch 35: 0.431550
loss in epoch 36: 0.433477
loss 

### 2.5 保存模型

In [10]:
# save model
targeted_model_file_name = './MNIST_target_model.pth'
torch.save(target_model.state_dict(), targeted_model_file_name)
target_model.eval()

MNIST_target_net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (logits): Linear(in_features=200, out_features=10, bias=True)
)

### 2.6 测试目标模型

In [11]:
# MNIST test dataset
mnist_dataset_test = torchvision.datasets.MNIST('./dataset', 
                                                train=False, 
                                                transform=transforms.ToTensor(), 
                                                download=False)
test_dataloader = DataLoader(mnist_dataset_test, 
                             batch_size=model_batch_size, 
                             shuffle=True, 
                             num_workers=1)
num_correct = 0
for i, data in enumerate(test_dataloader, 0):
    test_img, test_label = data
    test_img, test_label = test_img.to(device), test_label.to(device)
    pred_lab = torch.argmax(target_model(test_img), 1)
    num_correct += torch.sum(pred_lab==test_label,0)

print('accuracy in testing set: %f\n'%(num_correct.item()/len(mnist_dataset_test)))

accuracy in testing set: 0.993100



## 3. AdvGAN的生成

### 3.1 定义判别器

In [0]:
class Discriminator(nn.Module):
    def __init__(self, image_nc):
        super(Discriminator, self).__init__()
        # MNIST: 1*28*28
        model = [
                nn.Conv2d(image_nc, 8, kernel_size=4, stride=2, padding=0, bias=True),
                nn.LeakyReLU(0.2),
                # 8*13*13
                nn.Conv2d(8, 16, kernel_size=4, stride=2, padding=0, bias=True),
                nn.BatchNorm2d(16),
                nn.LeakyReLU(0.2),
                # 16*5*5
                nn.Conv2d(16, 32, kernel_size=4, stride=2, padding=0, bias=True),
                nn.BatchNorm2d(32),
                nn.LeakyReLU(0.2),
                nn.Conv2d(32, 1, 1),
                nn.Sigmoid()
                # 32*1*1
        ]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        output = self.model(x).squeeze()
        return output

### 3.2  定义resnet block

In [0]:
# modified from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/models/networks.py
class ResnetBlock(nn.Module):
    def __init__(self, dim, padding_type='reflect', norm_layer=nn.BatchNorm2d, use_dropout=False, use_bias=False):
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        conv_block = []
        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias),
                       norm_layer(dim),
                       nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias),
                       norm_layer(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x)
        return out


### 3.3 定义生成器

In [0]:
class Generator(nn.Module):
    def __init__(self,
                 gen_input_nc,
                 image_nc,
                ):
        super(Generator, self).__init__()

        encoder_lis = [
                      # MNIST:1*28*28
                      nn.Conv2d(gen_input_nc, 8, kernel_size=3, stride=1, padding=0, bias=True),
                      nn.InstanceNorm2d(8),
                      nn.ReLU(),
                      # 8*26*26
                      nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=0, bias=True),
                      nn.InstanceNorm2d(16),
                      nn.ReLU(),
                      # 16*12*12
                      nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0, bias=True),
                      nn.InstanceNorm2d(32),
                      nn.ReLU(),
                      # 32*5*5
        ]

        bottle_neck_lis = [ResnetBlock(32),
                           ResnetBlock(32),
                           ResnetBlock(32),
                           ResnetBlock(32),]

        decoder_lis = [
                      nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=0, bias=False),
                      nn.InstanceNorm2d(16),
                      nn.ReLU(),
                      # state size. 16 x 11 x 11
                      nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=0, bias=False),
                      nn.InstanceNorm2d(8),
                      nn.ReLU(),
                      # state size. 8 x 23 x 23
                      nn.ConvTranspose2d(8, image_nc, kernel_size=6, stride=1, padding=0, bias=False),
                      nn.Tanh()
                      # state size. image_nc x 28 x 28
        ]

        self.encoder = nn.Sequential(*encoder_lis)
        self.bottle_neck = nn.Sequential(*bottle_neck_lis)
        self.decoder = nn.Sequential(*decoder_lis)

    def forward(self, x):
        x = self.encoder(x)
        x = self.bottle_neck(x)
        x = self.decoder(x)
        return x

### 3.4 训练AdvGAN_Attack

In [0]:
import numpy as np
import os

models_path = './models/'

#### 3.4.1 分别初始化D权值和G的

In [0]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

#### 3.4.2 定义AdvGAN_Attack这个类

In [0]:
class AdvGAN_Attack:
    def __init__(self,
                 device,
                 model,
                 model_num_labels,
                 image_nc,
                 box_min,
                 box_max):
        output_nc = image_nc
        self.device = device
        self.model_num_labels = model_num_labels
        self.model = model
        self.input_nc = image_nc
        self.output_nc = output_nc
        self.box_min = box_min
        self.box_max = box_max

        self.gen_input_nc = image_nc
        self.netG = Generator(self.gen_input_nc, image_nc).to(device)
        self.netDisc = Discriminator(image_nc).to(device)

        # initialize all weights
        self.netG.apply(weights_init)
        self.netDisc.apply(weights_init)

        # initialize optimizers
        self.optimizer_G = torch.optim.Adam(self.netG.parameters(),
                                            lr=0.001)
        self.optimizer_D = torch.optim.Adam(self.netDisc.parameters(),
                                            lr=0.001)

        if not os.path.exists(models_path):
            os.makedirs(models_path)

    def train_batch(self, x, labels):
        # optimize D
        for i in range(1):
            perturbation = self.netG(x)

            # add a clipping trick
            adv_images = torch.clamp(perturbation, -0.3, 0.3) + x
            adv_images = torch.clamp(adv_images, self.box_min, self.box_max)

            self.optimizer_D.zero_grad()
            pred_real = self.netDisc(x)
            loss_D_real = F.mse_loss(pred_real, torch.ones_like(pred_real, device=self.device))
            loss_D_real.backward()

            pred_fake = self.netDisc(adv_images.detach())
            loss_D_fake = F.mse_loss(pred_fake, torch.zeros_like(pred_fake, device=self.device))
            loss_D_fake.backward()
            loss_D_GAN = loss_D_fake + loss_D_real
            self.optimizer_D.step()

        # optimize G
        for i in range(1):
            self.optimizer_G.zero_grad()

            # cal G's loss in GAN
            pred_fake = self.netDisc(adv_images)
            loss_G_fake = F.mse_loss(pred_fake, torch.ones_like(pred_fake, device=self.device))
            loss_G_fake.backward(retain_graph=True)

            # calculate perturbation norm
            C = 0.1
            loss_perturb = torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1))
            # loss_perturb = torch.max(loss_perturb - C, torch.zeros(1, device=self.device))

            # cal adv loss
            logits_model = self.model(adv_images)
            probs_model = F.softmax(logits_model, dim=1)
            onehot_labels = torch.eye(self.model_num_labels, device=self.device)[labels]

            # C&W loss function
            real = torch.sum(onehot_labels * probs_model, dim=1)
            other, _ = torch.max((1 - onehot_labels) * probs_model - onehot_labels * 10000, dim=1)
            zeros = torch.zeros_like(other)
            loss_adv = torch.max(real - other, zeros)
            loss_adv = torch.sum(loss_adv)

            # maximize cross_entropy loss
            # loss_adv = -F.mse_loss(logits_model, onehot_labels)
            # loss_adv = - F.cross_entropy(logits_model, labels)

            adv_lambda = 10
            pert_lambda = 1
            loss_G = adv_lambda * loss_adv + pert_lambda * loss_perturb
            loss_G.backward()
            self.optimizer_G.step()

        return loss_D_GAN.item(), loss_G_fake.item(), loss_perturb.item(), loss_adv.item()

    def train(self, train_dataloader, epochs):
        for epoch in range(1, epochs+1):

            if epoch == 50:
                self.optimizer_G = torch.optim.Adam(self.netG.parameters(),
                                                    lr=0.0001)
                self.optimizer_D = torch.optim.Adam(self.netDisc.parameters(),
                                                    lr=0.0001)
            if epoch == 80:
                self.optimizer_G = torch.optim.Adam(self.netG.parameters(),
                                                    lr=0.00001)
                self.optimizer_D = torch.optim.Adam(self.netDisc.parameters(),
                                                    lr=0.00001)
            loss_D_sum = 0
            loss_G_fake_sum = 0
            loss_perturb_sum = 0
            loss_adv_sum = 0
            for i, data in enumerate(train_dataloader, start=0):
                images, labels = data
                images, labels = images.to(self.device), labels.to(self.device)

                loss_D_batch, loss_G_fake_batch, loss_perturb_batch, loss_adv_batch = \
                    self.train_batch(images, labels)
                loss_D_sum += loss_D_batch
                loss_G_fake_sum += loss_G_fake_batch
                loss_perturb_sum += loss_perturb_batch
                loss_adv_sum += loss_adv_batch

            # print statistics
            num_batch = len(train_dataloader)
            print("epoch %d:\nloss_D: %.3f, loss_G_fake: %.3f,\
             \nloss_perturb: %.3f, loss_adv: %.3f, \n" %
                  (epoch, loss_D_sum/num_batch, loss_G_fake_sum/num_batch,
                   loss_perturb_sum/num_batch, loss_adv_sum/num_batch))

            # save generator
            if epoch%20==0:
                netG_file_name = models_path + 'netG_epoch_' + str(epoch) + '.pth'
                torch.save(self.netG.state_dict(), netG_file_name)

#### 3.4.3 开始训练

In [0]:
epochs = 60
AdvGAN_batch_size = 128
BOX_MIN = 0
BOX_MAX = 1

In [0]:
pretrained_model = "./MNIST_target_model.pth"
targeted_model = MNIST_target_net().to(device)
targeted_model.load_state_dict(torch.load(pretrained_model))
targeted_model.eval()
model_num_labels = 10

In [20]:
# MNIST train dataset and dataloader declaration
mnist_dataset = torchvision.datasets.MNIST('./dataset', 
                                           train=True, 
                                           transform=transforms.ToTensor(), 
                                           download=False)
dataloader = DataLoader(mnist_dataset, 
                        batch_size=AdvGAN_batch_size, 
                        shuffle=True, 
                        num_workers=1)
advGAN = AdvGAN_Attack(device,
                          targeted_model,
                          model_num_labels,
                          image_nc,
                          BOX_MIN,
                          BOX_MAX)

advGAN.train(dataloader, epochs)

epoch 1:
loss_D: 0.249, loss_G_fake: 0.482,             
loss_perturb: 16.324, loss_adv: 41.130, 

epoch 2:
loss_D: 0.049, loss_G_fake: 0.767,             
loss_perturb: 17.129, loss_adv: 15.665, 

epoch 3:
loss_D: 0.018, loss_G_fake: 0.860,             
loss_perturb: 17.338, loss_adv: 8.594, 

epoch 4:
loss_D: 0.007, loss_G_fake: 0.914,             
loss_perturb: 17.496, loss_adv: 5.720, 

epoch 5:
loss_D: 0.006, loss_G_fake: 0.930,             
loss_perturb: 17.463, loss_adv: 4.802, 

epoch 6:
loss_D: 0.003, loss_G_fake: 0.948,             
loss_perturb: 17.250, loss_adv: 3.904, 

epoch 7:
loss_D: 0.001, loss_G_fake: 0.962,             
loss_perturb: 16.961, loss_adv: 2.978, 

epoch 8:
loss_D: 0.002, loss_G_fake: 0.965,             
loss_perturb: 16.953, loss_adv: 2.632, 

epoch 9:
loss_D: 0.001, loss_G_fake: 0.969,             
loss_perturb: 16.780, loss_adv: 2.499, 

epoch 10:
loss_D: 0.001, loss_G_fake: 0.970,             
loss_perturb: 16.804, loss_adv: 2.365, 

epoch 11:
loss_D:

## 4 测试对抗样本

In [0]:
image_nc = 1
test_batch_size = 128

gen_input_nc = image_nc

In [22]:
# load the pretrained model
pretrained_model = "./MNIST_target_model.pth"
target_model = MNIST_target_net().to(device)
target_model.load_state_dict(torch.load(pretrained_model))
target_model.eval()

MNIST_target_net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (logits): Linear(in_features=200, out_features=10, bias=True)
)

In [24]:
# load the generator of adversarial examples
pretrained_generator_path = './models/netG_epoch_60.pth'
pretrained_G = Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

Generator(
  (encoder): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU()
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2))
    (4): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU()
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (7): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (8): ReLU()
  )
  (bottle_neck): Sequential(
    (0): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (6): 

In [26]:
# test adversarial examples in MNIST training dataset
mnist_dataset = torchvision.datasets.MNIST('./dataset', train=True, transform=transforms.ToTensor(), download=True)
train_dataloader = DataLoader(mnist_dataset, batch_size=test_batch_size, shuffle=False, num_workers=1)
num_correct = 0
for i, data in enumerate(train_dataloader, 0):
    test_img, test_label = data
    test_img, test_label = test_img.to(device), test_label.to(device)
    perturbation = pretrained_G(test_img)
    perturbation = torch.clamp(perturbation, -0.3, 0.3)
    adv_img = perturbation + test_img
    adv_img = torch.clamp(adv_img, 0, 1)
    pred_lab = torch.argmax(target_model(adv_img),1)
    num_correct += torch.sum(pred_lab==test_label,0)

    
print('MNIST training dataset:')
print('num_correct: ', num_correct.item())
print('accuracy of adv imgs in training set: %f\n'%(num_correct.item()/len(mnist_dataset)))

MNIST training dataset:
num_correct:  198
accuracy of adv imgs in training set: 0.003300



In [28]:
# test adversarial examples in MNIST testing dataset
mnist_dataset_test = torchvision.datasets.MNIST('./dataset', train=False, transform=transforms.ToTensor(), download=True)
test_dataloader = DataLoader(mnist_dataset_test, batch_size=test_batch_size, shuffle=False, num_workers=1)
num_correct = 0
for i, data in enumerate(test_dataloader, 0):
    test_img, test_label = data
    test_img, test_label = test_img.to(device), test_label.to(device)
    perturbation = pretrained_G(test_img)
    perturbation = torch.clamp(perturbation, -0.3, 0.3)
    adv_img = perturbation + test_img
    adv_img = torch.clamp(adv_img, 0, 1)
    pred_lab = torch.argmax(target_model(adv_img),1)
    num_correct += torch.sum(pred_lab==test_label,0)

print('num_correct: ', num_correct.item())
print('accuracy of adv imgs in testing set: %f\n'%(num_correct.item()/len(mnist_dataset_test)))

num_correct:  58
accuracy of adv imgs in testing set: 0.005800

